In [1]:
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.recurrent import LSTM
from keras.models import Sequential
import lstm, time #helper libraries
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import detrendPrice

Using TensorFlow backend.


In [2]:
pd.core.common.is_list_like = pd.api.types.is_list_like #datareader problem probably fixed in next version of datareader
from pandas_datareader import data as pdr
import datetime
#import numpy as np

import fix_yahoo_finance as yf
yf.pdr_override() # <== that's all it takes :-)


start_date=datetime.date(1980, 1, 1)
end_date= datetime.date(2018, 1, 1)

stock_list = ["AAPL"]

stock_str = ""
for i in range(len(stock_list)):
    stock_str  = stock_str + stock_list[i] + "."

main_df = pd.DataFrame()

for stock in range(len(stock_list)):
     df = pdr.get_data_yahoo(stock_list[stock], start=start_date, end=end_date)
     df.head()
     #df.drop(['Close','High', 'Low' , 'Open', 'Volume'], axis=1, inplace=True)
     df.rename(columns={'Adj Close': 'Adj_close'}, inplace=True)
     if main_df.empty:
         main_df = df
     else:
        main_df = main_df.join(df) 
    
#main_df["Date"]=main_df.index
main_df.reset_index(level=0, inplace=True)
main_df.head()
df = main_df

[*********************100%***********************]  1 of 1 downloaded


In [3]:
df = main_df.assign(log_return = np.log(main_df['Adj_close'])-np.log(main_df['Adj_close']).shift(1))
df['weekly_mean'] = df['log_return'].rolling(window=5).mean()
df['biweekly_mean'] = df['log_return'].rolling(window=10).mean()
df['monthly_mean'] = df['log_return'].rolling(window=21).mean()
df['bimonthly_mean'] = df['log_return'].rolling(window=42).mean()
df['semiannual_mean'] = df['log_return'].rolling(window=126).mean()
df['annual_mean'] = df['log_return'].rolling(window=252).mean()
df['weekly_std'] = df['log_return'].rolling(window=5).std()
df['biweekly_std'] = df['log_return'].rolling(window=10).std()
df['monthly_std'] = df['log_return'].rolling(window=21).std()
df['bimonthly_std'] = df['log_return'].rolling(window=42).std()
df['semiannual_std'] = df['log_return'].rolling(window=126).std()
df['annual_std'] = df['log_return'].rolling(window=252).std()
df['weekly_volume_mean'] = df['Volume'].rolling(window=5).mean()
df['biweekly_volume_mean'] = df['Volume'].rolling(window=10).mean()
df['monthly_volume_mean'] = df['Volume'].rolling(window=21).mean()
df['bimonthly_volume_mean'] = df['Volume'].rolling(window=42).mean()
df['semiannual_volume_mean'] = df['log_return'].rolling(window=126).mean()
df['annual_volume_mean'] = df['log_return'].rolling(window=252).mean()
df['weekly_volume_std'] = df['Volume'].rolling(window=5).std()
df['biweekly_volume_std'] = df['Volume'].rolling(window=10).std()
df['monthly_volume_std'] = df['Volume'].rolling(window=21).std()
df['bimonthly_volume_std'] = df['Volume'].rolling(window=42).std()
df['semiannual_volume_std'] = df['log_return'].rolling(window=126).std()
df['annual_volume_std'] = df['log_return'].rolling(window=252).std()

df = df.dropna()
df.reset_index(level=0, inplace=True,drop=True)
df.head(50)


,Date,Open,High,Low,Close,Adj_close,Volume,log_return,weekly_mean,biweekly_mean,...,monthly_volume_mean,bimonthly_volume_mean,semiannual_volume_mean,annual_volume_mean,weekly_volume_std,biweekly_volume_std,monthly_volume_std,bimonthly_volume_std,semiannual_volume_std,annual_volume_std
0,1981-12-14,0.328125,0.328125,0.323661,0.323661,0.014504,6311200,-0.033959,-0.010751,-0.002727,...,1.007840e+07,9.450667e+06,-0.004604,-0.001831,4.938716e+06,8.787443e+06,6.610926e+06,5.574471e+06,0.034105,0.034049
1,1981-12-15,0.332589,0.334821,0.332589,0.332589,0.014905,7828800,0.027272,-0.001337,0.000000,...,1.020107e+07,9.461867e+06,-0.004233,-0.001510,5.049142e+06,8.640881e+06,6.540432e+06,5.570636e+06,0.034196,0.033941
2,1981-12-16,0.348214,0.350446,0.348214,0.348214,0.015605,16363200,0.045895,0.006513,0.003921,...,1.071173e+07,9.634400e+06,-0.003743,-0.001025,5.587587e+06,8.622361e+06,6.584945e+06,5.671001e+06,0.034470,0.033740
3,1981-12-17,0.377232,0.379464,0.377232,0.377232,0.016905,12863200,0.080018,0.022517,0.013264,...,1.090267e+07,9.818133e+06,-0.003076,-0.000805,5.425110e+06,8.124785e+06,6.586503e+06,5.647006e+06,0.035268,0.034087
4,1981-12-18,0.408482,0.410714,0.408482,0.408482,0.018306,17931200,0.079619,0.039769,0.018560,...,1.140960e+07,1.003240e+07,-0.002218,-0.000602,5.108482e+06,4.446821e+06,6.702839e+06,5.781687e+06,0.035953,0.034413
5,1981-12-21,0.392857,0.392857,0.390625,0.390625,0.017505,14100800,-0.044742,0.037612,0.013431,...,1.160480e+07,9.910400e+06,-0.002272,-0.001015,3.881320e+06,4.411917e+06,6.719453e+06,5.635191e+06,0.036012,0.034316
6,1981-12-22,0.397321,0.399554,0.397321,0.397321,0.017805,13456800,0.016993,0.035557,0.017110,...,1.179200e+07,1.003867e+07,-0.002272,-0.001136,2.132485e+06,4.422323e+06,6.713389e+06,5.653551e+06,0.036012,0.034197
7,1981-12-23,0.390625,0.390625,0.388393,0.388393,0.017405,7224000,-0.022722,0.021833,0.014173,...,1.186267e+07,1.004453e+07,-0.002249,-0.001390,3.842255e+06,4.575563e+06,6.654047e+06,5.650424e+06,0.035998,0.034118
8,1981-12-24,0.390625,0.392857,0.390625,0.390625,0.017505,7229600,0.005729,0.006975,0.014746,...,1.194320e+07,1.005427e+07,-0.002373,-0.001571,4.671215e+06,4.779669e+06,6.583539e+06,5.645085e+06,0.035943,0.033958
9,1981-12-28,0.377232,0.377232,0.372768,0.372768,0.016705,9144800,-0.046778,-0.018304,0.010732,...,1.175307e+07,9.762533e+06,-0.002643,-0.002107,3.339589e+06,4.206912e+06,6.604941e+06,5.353658e+06,0.036148,0.033598


In [4]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
le.fit(pd.to_datetime(df['Date']).dt.day_name())
df['day'] = le.transform(pd.to_datetime(df['Date']).dt.day_name())
le = LabelEncoder()
le.fit(pd.to_datetime(df['Date']).dt.week)
df['week'] = le.transform(pd.to_datetime(df['Date']).dt.week)
le = LabelEncoder()
le.fit(pd.to_datetime(df['Date']).dt.month)
df['month'] = le.transform(pd.to_datetime(df['Date']).dt.month)

In [5]:
from sklearn.preprocessing import LabelBinarizer
lb = LabelBinarizer()
lb.fit(df['day'])
encoded_days = lb.transform(df['day'])
df_temp = pd.DataFrame(encoded_days, columns=['Day'+str(x) for x in range(0,encoded_days.shape[1])])
df = pd.concat([df, df_temp], axis=1)
lb.fit(df['week'])
encoded_weeks = lb.transform(df['week'])
df_temp = pd.DataFrame(encoded_weeks, columns= ['Week'+str(x) for x in range(0,encoded_weeks.shape[1])])
df = pd.concat([df, df_temp], axis=1)
lb.fit(df['month'])
encoded_months = lb.transform(df['month'])
df_temp = pd.DataFrame(encoded_months, columns= ['Month'+str(x) for x in range(0,encoded_months.shape[1])])
df = pd.concat([df, df_temp], axis=1)

In [6]:
df1 = df.pop('Adj_close')
df['Adj_close'] = df1
df.to_csv('AAPL.csv')